In [ ]:
#Linear model 
---
title: "Untitled"
author: "Lindani Sumaili"
date: "25/04/2021"
output: html_document
---

```{r setup, include=FALSE}
library(bnlearn)
library(Rgraphviz)
library(gRain)
library(dplyr)
library(GGally)
library(ggplot2)
library(caret)
library(e1071)
library(tidyverse)
#install.packages("ggthemes")
library(ggthemes)
#install.packages("MLmetrics")
library(MLmetrics)
```

```{r}
#upload data
library(readr)
heart_BN <- read_csv("C:/Users/linda/OneDrive/Documents/R/heart.csv")
```
```{r}
#Rename columns
heart_BN <- heart_BN %>% rename(RestBloodPres = trtbps, MaxHeartRt = thalachh, ChestPainTy = cp,FastBloodSug = fbs, RestECG = restecg, 
                                ExInducedAng = exng , SerumChol= chol, DepInduEx= oldpeak, SlpEX = slp, VesColored= caa, DefectTy= thall, HrtDisease = output )
```

```{r}
#calculate missing values
heart_BN %>%
  select(everything()) %>%  # replace to your needs
  summarise_all(funs(sum(is.na(.))))

```

```{r fig.align="center", echo = FALSE,fig.width = 18}
#visualize the missing data
plot2 = ggplot2::theme_set(ggplot2::theme_bw())  ## (this just globally sets the ggplot2 theme to theme_bw) 
heart_BN %>% select(-HrtDisease) %>% ggpairs() %>%

print(progress = FALSE)

``` 

```{r fig.align="center", echo = FALSE,fig.width = 14}
# visualization of correlations
ggcorr(heart_BN, method = c("everything", "pearson")) 
```
```{r}
#linear model
lm2 = lm(HrtDisease~., data = heart_BN) #Create a linear regression with two variables
summary(lm2) #Review the results

```

```{r}
plot(lm2$residuals,col = "blue")
```

```{r}
plot(cooks.distance(lm2),pch = 16, col = "blue") #Plot the Cooks Distances.
```

```{r}
#Logistic linear model 
colnames(heart_BN)[14] ="Res"
heart_BN$Res = as.factor(heart_BN$Res)
heart = caret::downSample(x=heart_BN[,1:13],y=heart_BN$Res, yname = "Res")

dt = sort(sample(nrow(heart), nrow(heart)*.7))

train = heart[dt,]
test = heart[-dt,]

#train = train[,-14]
#test = test[,-14]

m1 = glm(Res ~ ., family = "binomial", data=train) # glm fit generalized linear model
summary(m1)
#model <- glm(formula(train), train, family = "binomial")
#model
preds = predict(m1, newdata = test, type = "response")
auc1 = MLmetrics::AUC(preds,test$Res) #compute the Area Under Operating Characteristic Curve(ROC AUC)from prediction scores
auc1 
#auc1 = 0.9360465
mean(auc1)

```

```{r}
Precision(test$Res, ifelse(preds >= .5, 1, 0), positive = 1) # 0.9114219
#
f1_scores <- sapply(seq(0.01, 0.99, .01), function(thresh) F1_Score(test$Res, ifelse(preds >= thresh, 1, 0), positive = 1))

which.max(f1_scores) # 32
#
Precision(test$Res, ifelse(preds >= .32, 1, 0), positive = 1)

Recall(test$Res, ifelse(preds >= .32, 1, 0), positive = 1)

Accuracy(ifelse(preds >= .32, 1, 0), test$Res)

```

In [ ]:
#Naive Bayes
---
title: "Untitled"
author: "Lindani Sumaili"
date: "18/04/2021"
output: html_document
---

```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
```

```{r}
#install.packages("readxl")
#install.packages("dplyr")
#install.packages("GGally")
#install.packages("ggplot2")
#install.packages('caret')
#install.packages("e1071") #Holds the Naive Bayes function
#install.packages('tidyverse')

```

Note that the `echo = FALSE` parameter was added to the code chunk to prevent printing of the R code that generated the plot.
```{r include=FALSE}
#Loading required packages
library(readxl)
library(dplyr)
library(GGally)
library(ggplot2)
library(caret)
library(e1071)
library(tidyverse)


```

```{r}
heart <- read.csv("heart.csv", header = TRUE) #Upload the data
heart <- heart %>% rename(RestBloodPres = trtbps, MaxHeartRt = thalachh, ChestPainTy = cp,FastBloodSug = fbs, RestECG = restecg, 
                                ExInducedAng = exng , SerumChol= chol, DepInduEx= oldpeak, SlpEX = slp, VesColored= caa, 
                                DefectTy= thall,HrtDisease = output )

head(heart,5)

```

```{r}
#Boxplot to see the outliers 
X = heart[,1:4] 

for (i in 1:4){
  # Histogram
  hist(heart[,i], probability = TRUE, col = "grey", axes = FALSE, main = "",
       xlab = colnames(X[i]) )
  
  # Axis
  #axis(1)

  # Density
  lines(density(heart[,i]), col = "red", lwd = 2)
  
  par(new = TRUE)
  
  boxplot(heart[,i], horizontal = TRUE, axes = TRUE,
        lwd = 2, col = rgb(0, 1, 1, alpha = 0.15))

}

```

```{r}
colSums(is.na(heart))
#print('----------------------')
unique =heart[,5:10]
for (i in 5:10)
  print(heart %>% count(heart[i]))
```

```{r}

```


```{r}
str(heart)
```

```{r setup, include=TRUE}
#visualize the missing data
ggplot2::theme_set(ggplot2::theme_bw())  ## (this just globally sets the ggplot2 theme to theme_bw) 
heart %>% select(-output) %>% ggpairs() %>%
print(progress = F)

#ggpairs(heart,title="correlogram with ggpairs()")

```

```{r}
# Nice visualization of correlations
ggcorr(heart, method = c("everything", "pearson")) 

```

```{r}
#Building a model
#Setting outcome variables as categorical
heart$HrtDisease <- factor(heart$HrtDisease, levels = c(0,1), labels = c("False", "True"))

```

```{r}
#split data into training and test data sets
indxTrain <- createDataPartition(y = heart$HrtDisease,p = 0.75,list = FALSE)
training <- heart[indxTrain,]
testing <- heart[-indxTrain,] #Check dimensions of the split > prop.table(table(data$Outcome)) * 100
prop.table(table(training$HrtDisease)) * 100
prop.table(table(testing$HrtDisease)) * 100

```

```{r}
#create objects x which holds the predictor variables and y which holds the response variables
x = training[,-14]
y = training$HrtDisease

```

```{r echo = FALSE, warning=FALSE}
model = train(x,y,'nb',trControl=trainControl(method='cv',number=10))
model 

```

```{r echo = FALSE, warning=FALSE}
#Model Evaluation
#Predict testing set
Predict <- predict(model,newdata = testing ) #Get the confusion matrix to see Confusion Matrix and Statistics
confusionMatrix(Predict, testing$HrtDisease )

```

```{r}
#Plot Variable performance
X <- varImp(model)
plot(X, main = "Variable importance to predict the outcome    ")
```


In [ ]:
#Bayesian Network 
---
title: "Untitled"
author: "Lindani Sumaili"
date: "03/05/2021"
output: html_document
---

```{r setup, include=FALSE}
library(Rgraphviz)
library(gRain)
library(dplyr)
library(GGally)
library(ggplot2)
library(caret)
library(e1071)
library(tidyverse)
library(forecast)
library(bnlearn)
library(gRbase)
library(grid)
#install.packages("bnlearn")
```

```{r setup, include=FALSE}
#upload data
library(readr)
heart_BN1 <- read_csv("C:/Users/linda/OneDrive/Documents/R/heart.csv")
heart_BN2 <- read_csv("C:/Users/linda/OneDrive/Documents/R/heart.csv")
```


```{r}
#Rename columns for heart_BN1
heart_BN1 <- heart_BN1 %>% rename(RestBloodPres = trtbps, MaxHeartRt = thalachh, ChestPainTy = cp,FastBloodSug = fbs, RestECG = restecg, 
                                ExInducedAng = exng , SerumChol= chol, DepInduEx= oldpeak, SlpEX = slp, VesColored= caa, DefectTy= thall, HrtDisease = output )
#Rename columns for heart_BN2
heart_BN2 <- heart_BN2 %>% rename(a = trtbps, b = thalachh, c = cp, d = fbs, e = restecg, 
                                f = exng , g = chol, h = oldpeak, i = slp, j = caa, k = thall, l = output )

```


```{r}
# learn the network structure.
#bn.fit: fit the parameters of a Bayesian network 
#In bnlearn: Bayesian Network Structure Learning, Parameter learning and Inference
res = hc(heart_BN2) #learn the structure of BN
res
print("-----------------------------------------------------------")
res2 <- gs(heart_BN2)
res2
#print("--------Parameters Estimate using  Hill Climb -----------------------")
# estimate the parameters of the Bayesian network.mle:Maximum Likelihood parameter estimation
fitted = bn.fit(res, heart_BN2, method = "mle")
#fitted
print("-Find the equivalence class and the v-structures of a Bayesian network----------")
cpdag(res) #categorical Bayesian Network inference
#dag2cpdag(res)
print("----------------------------------------------------------------")
vstructs(res ) # parents are linked by an arc
#print("----------------------------------------------------------------")
#cpdag(res2)
#vstructs(res2)
```

```{r}
print("--------Parameters Estimate using  Hill Climb -----------------------")
# estimate the parameters of the Bayesian network.mle:Maximum Likelihood parameter estimation
#fitted = bn.fit(res, heart_BN2, method = "mle")
fitted
```



```{r fig.align="center", echo = FALSE, fig.width = 10}
graphviz.plot(res, layout = "dot", shape = "circle", main = "Score-based methods: Hill Climb Algorithm    ", render = TRUE )

#graphviz.plot(res2, main = "  categorical Bayesian Network inference   ")
graphviz.plot(cpdag(res), layout = "dot", shape = "circle", main = "Cpdag Score-based methods: Hill Climb Algorithm    ", render = TRUE )
```

```{r}
## Gaussian Bayesian network.
#cpquery: Perform conditional probability queries (CPQs).
fitted2 = bn.fit(hc(heart_BN2), heart_BN2)
# the result should be around 0.04.
cpquery(fitted2,
  event = ((A >= 0) & (A <= 1)) & ((B >= 0) & (B <= 3)),
  evidence = (C + D < 10))

## ideal interventions and mutilated networks.
mutilated(fitted, evidence = list(F = 42))


```

```{r}
#AIC
#Inspect distributions
hipcenter_mod = lm(l ~ ., data = heart_BN2)
coef(hipcenter_mod)

```

```{r}
hip_back_aic = step(hipcenter_mod, direction = "backward")
coef(hip_back_aic)
```

```{r}
#Cross-validation for Bayesian network
bn.cv(heart_BN2, 'mmhc')
```

```{r}
# conditional independence test to check 
ci.test(heart_BN2)


```


```{r}
#Regression metrics 
dt = sort(sample(nrow(heart_BN2), nrow(heart_BN2)*.7)) #select dat at random 70/30
train.set = heart_BN2[dt,]
test.set = heart_BN2[-dt,]

res3 = hc(train.set)                 # learn BN structure on training set data
res3
fitted = bn.fit(res3, train.set)     # learning of parameters
#fitted

pred = predict(fitted,"l", test.set)  # predicts the value of node l given test set
cbind(pred, test.set[,"l"])           # compare the actual and predicted
print("---------------------------------------------------------------------")
print("         Metrics Achived - heart disease")
accuracy(object = pred, x = test.set$"l")

pred2 = predict(fitted,"c", test.set)  # predicts the value of node l given test set
cbind(pred2, test.set[,"c"])           # compare the actual and predicted
print("---------------------------------------------------------------------")
print("         Metrics Achived - Chest Pain type")
accuracy(object = pred2, x = test.set$"c")

```


```{r}


```

```{r}
graphviz.plot(res3, layout = "dot", shape = "circle", main = "Score-based methods: Hill Climb Algorithm    ", render = TRUE )
```


```{r}
cpdag(res3)
print("----------------------------------------------------------------")

vstructs(res3)


```